In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


1. **라이브러리 및 데이터 불러오기
 - **라이브러리 버전 확인 반드시 해준다!!**
2. 탐색적 자료 분석 (EDA)
 - 데이터 시각화
 - 산점도, 막대그래프 등
 - 그래프 해석해서 설명 달기
 - 약간의 데이터 전처리
3. 데이터 전처리
 - Feature Engineering
 - ML 모형을 돌리기 위해 표준화 등/ 원핫-인코딩
 - 파생변수(도출변수) 만들기
  + 왜 이변수를 만들었는지에 대한 설명 필요
4. 머신러닝 모형 개발
 - 모형(로지스틱,,등)에 대한 설명 필요
 - 모형은 1, 2개 정도만 사용하자
 - 교차 검증, 하이퍼파라미터 튜닝
5. 모형 평가
 - 훈련 데이터를 쪼개서 훈련 데이터와 검증 데이터 분리
 - 정확도 비교
 - 혼동행렬(confusion Matrix) 설명을 잘 찾아서 넣어주자
6. 제출
 - 제출 양식 샘플은 쌤이 만들어 준다.
★. 참고
 - 다른 사람의 code를 참고 했다면 노트북 표절 방지를 위해 참조한 코드는 반드시 링크 걸어둘 것
  + 저자 이름, 글 제목, 링크 주소.
★. 마감일
 - 4월 12일 17시 40분
 - 제출 형태
  + Leaderboard 랭킹 사진 캡쳐
  + 고용노동부 보고하기.

★★ 코드가 오류가 계속 날 때는 쌤이 도움이 되는 링크만 준다(실력은 늘지도,,)

#### 데이터 설명
- 우주선 타이타닉이 시공간 이상과 충돌 하는 동안에 승객이 다른 차원으로 이동했는가에 대한 여부 예측하는 것.
 + 이러한 예측을 돕기 위해 손상된 함선의 컴퓨터 시스템에서 복구된 개인 기록이 제공됩니다.

- sample_submission.csv : 올바른 형식의 송신 파일.
- train : 승객이 다른 차원으로 운송되었는지 여부. 예측하려는 대상 열입니다.(훈련데이터로 사용됨)
- test : 테스트 데이터로 사용되는 나머지 승객의 1/3(~4300)에 대한 개인 기록. 작업은 이 세트의 승객에 대한 운송 값을 예측하는 것

- PassengerId - 테스트 세트에 포함된 각 승객의 ID
- Transported - 타겟. 각 승객에 대해 참 또는 거짓을 예측

In [2]:
import pandas as pd
submission = pd.read_csv("../input/spaceship-titanic/sample_submission.csv")
test = pd.read_csv("../input/spaceship-titanic/test.csv")
df = pd.read_csv("../input/spaceship-titanic/train.csv")

In [3]:
submission.head()

,PassengerId,Transported
0,0013_01,False
1,0018_01,False
2,0019_01,False
3,0021_01,False
4,0023_01,False


In [4]:
test.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


In [5]:
df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [7]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   4277 non-null   object 
 1   HomePlanet    4190 non-null   object 
 2   CryoSleep     4184 non-null   object 
 3   Cabin         4177 non-null   object 
 4   Destination   4185 non-null   object 
 5   Age           4186 non-null   float64
 6   VIP           4184 non-null   object 
 7   RoomService   4195 non-null   float64
 8   FoodCourt     4171 non-null   float64
 9   ShoppingMall  4179 non-null   float64
 10  Spa           4176 non-null   float64
 11  VRDeck        4197 non-null   float64
 12  Name          4183 non-null   object 
dtypes: float64(6), object(7)
memory usage: 434.5+ KB


In [8]:
import numpy as np
import matplotlib.pyplot as plt

In [9]:
test.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name'],
      dtype='object')

In [10]:
df.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported'],
      dtype='object')

- 드랍을 시키는 이유는 모형에 아무런 영향을 주지 않기 때문.
- 드랍은 지정된 행이나 열을 삭제하기 때문에 행축을 지정(axis= 0 또는 1 )

In [11]:
df = df.drop(['PassengerId', 'Cabin', 'Name'], axis=1)
df

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False
1,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True
2,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False
3,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False
4,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True
...,...,...,...,...,...,...,...,...,...,...,...
8688,Europa,False,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,False
8689,Earth,True,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,False
8690,Earth,False,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,True
8691,Europa,False,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,False


In [12]:
df.count() # 각 객체 별 수

HomePlanet      8492
CryoSleep       8476
Destination     8511
Age             8514
VIP             8490
RoomService     8512
FoodCourt       8510
ShoppingMall    8485
Spa             8510
VRDeck          8505
Transported     8693
dtype: int64

In [13]:
# 결측치 확인
#df['Transported'].isnull().sum()

In [14]:
#df['Transported'].unique()

In [15]:
df['Transported'].value_counts()

True     4378
False    4315
Name: Transported, dtype: int64

In [16]:
df['VIP'].value_counts()

False    8291
True      199
Name: VIP, dtype: int64

In [17]:
df_nottransported = df[(df["Transported"] == 0)]
df_nottransported.shape

(4315, 11)

In [18]:
df_transported = df[(df["Transported"] == 1)]
df_transported.shape

(4378, 11)

In [19]:
#fig, ax = plt.subplots()
#sns.countplot(x='VIP', hub='Transported', data=df,
#             palette="nipy_spectral",
#             order=df['VIP'].value_counts().index)
#plt.show()

결측치 수치(복붙)

In [20]:
null=pd.DataFrame(df.isnull().sum(),columns=["Null Values"])
null["% Missing Values"]=(df.isna().sum()/len(df)*100)
null = null[null["% Missing Values"] > 0]
null.style.background_gradient(cmap='ocean',low =0.2,high=0.1)


,Null Values,% Missing Values
HomePlanet,201,2.312205
CryoSleep,217,2.496261
Destination,182,2.093639
Age,179,2.059128
VIP,203,2.335212
RoomService,181,2.082135
FoodCourt,183,2.105142
ShoppingMall,208,2.392730
Spa,183,2.105142
VRDeck,188,2.162660


In [21]:
null_cat=null.select_dtypes(include='object')
null_cat.head(3)

""
HomePlanet
CryoSleep
Destination


In [22]:
#null=pd.DataFrame(test.isnull().sum(),columns=["Null Values"])
#null["% Missing Values"]=(df.isna().sum()/len(df)*100)
#null = null[null["% Missing Values"] > 0]
#null.style.background_gradient(cmap='ocean',low =0.2,high=0.1)

In [23]:
#df.corr()

In [24]:
test = test.drop(['PassengerId', 'Cabin', 'Name'], axis = 1)
test

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,Earth,True,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0
1,Earth,False,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0
2,Europa,True,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0
3,Europa,False,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0
4,Earth,False,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
4272,Earth,True,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,0.0,0.0
4273,Earth,False,TRAPPIST-1e,42.0,False,0.0,847.0,17.0,10.0,144.0
4274,Mars,True,55 Cancri e,NaN,False,0.0,0.0,0.0,0.0,0.0
4275,Europa,False,NaN,NaN,False,0.0,2680.0,0.0,0.0,523.0


In [25]:
# train
df_cat = df.select_dtypes(include='object')
df_cat.head()

,HomePlanet,CryoSleep,Destination,VIP
0,Europa,False,TRAPPIST-1e,False
1,Earth,False,TRAPPIST-1e,False
2,Europa,False,TRAPPIST-1e,True
3,Europa,False,TRAPPIST-1e,False
4,Earth,False,TRAPPIST-1e,False


In [26]:
# test
df_cat = test.select_dtypes(include = 'object')
df_cat.head()

,HomePlanet,CryoSleep,Destination,VIP
0,Earth,True,TRAPPIST-1e,False
1,Earth,False,TRAPPIST-1e,False
2,Europa,True,55 Cancri e,False
3,Europa,False,TRAPPIST-1e,False
4,Earth,False,TRAPPIST-1e,False


CryoSleep(저온수면) False 또는 True, 목적지, VIP 관계성 

- 간단한 통계
 + df(train), test

In [27]:
df_1 = df.describe()

In [28]:
df['VIP'].isnull().sum()

203

In [29]:
test.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,4186.000000,4195.000000,4171.000000,4179.000000,4176.000000,4197.000000
mean,28.658146,219.266269,439.484296,177.295525,303.052443,310.710031
std,14.179072,607.011289,1527.663045,560.821123,1117.186015,1246.994742
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,26.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,37.000000,53.000000,78.000000,33.000000,50.000000,36.000000
max,79.000000,11567.000000,25273.000000,8292.000000,19844.000000,22272.000000


In [30]:
df_data = df[['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].to_numpy()
print(df_data.shape)
print(df_data)

(8693, 6)
[[3.900e+01 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
 [2.400e+01 1.090e+02 9.000e+00 2.500e+01 5.490e+02 4.400e+01]
 [5.800e+01 4.300e+01 3.576e+03 0.000e+00 6.715e+03 4.900e+01]
 ...
 [2.600e+01 0.000e+00 0.000e+00 1.872e+03 1.000e+00 0.000e+00]
 [3.200e+01 0.000e+00 1.049e+03 0.000e+00 3.530e+02 3.235e+03]
 [4.400e+01 1.260e+02 4.688e+03 0.000e+00 0.000e+00 1.200e+01]]


In [31]:
df_target = df[['Transported']].to_numpy()
df_target

array([[False],
       [ True],
       [False],
       ...,
       [ True],
       [False],
       [ True]])

In [32]:
df = pd.DataFrame()

In [33]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(
    df_data, df_target, test_size = 0.2, random_state=42)

print(train_input.shape, test_input.shape)
print(train_input)

(6954, 6) (1739, 6)
[[2.800e+01 0.000e+00 5.500e+01 0.000e+00 6.560e+02 0.000e+00]
 [1.700e+01 0.000e+00 1.195e+03 3.100e+01 0.000e+00 0.000e+00]
 [2.800e+01 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
 ...
 [2.200e+01 1.580e+02 0.000e+00 4.760e+02 0.000e+00 2.600e+01]
 [3.400e+01 3.790e+02 0.000e+00 1.626e+03 0.000e+00 0.000e+00]
 [2.800e+01 7.000e+00 4.890e+02 0.000e+00 4.000e+00 6.027e+03]]


In [34]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

In [35]:
#from sklearn.linear_model import LogisticRegression
#lr = LogisticRegression()
#lr.fit(train_scaled, train_target)
#print(lr.score(train_scaled, train_target))
#print(lr.score(test_scaled, test_target))